# Проверка информативности признаков 3D моделей (STEP)

В этом ноутбуке мы проверим, насколько информативны извлекаемые признаки 3D моделей в формате STEP. Будет проведён анализ распределения признаков, их корреляции, а также оценена их значимость с помощью простых моделей машинного обучения.

## 1. Загрузка датасета и моделей

Загрузим датасет, получим список путей к моделям, реализуем функцию для загрузки shape из STEP-файла.

In [6]:
import notebook_setup
from src.dataset import DatasetIO
from src.config import INTERIM_DATA_DIR

import os
from tqdm import tqdm

# Загрузка датасета
pkl_file = INTERIM_DATA_DIR / "dataset_metadata.pkl"
dataset = DatasetIO.load_dataset_pickle(pkl_file)


print("Пример записи:", dataset[0])




2025-08-12 14:23:56.911 | INFO     | src.dataset:load_dataset_pickle:332 - Загрузка датасета из /home/developer/workspace/projects/3d_recognition_analisis/data/interim/dataset_metadata.pkl
2025-08-12 14:23:56.924 | SUCCESS  | src.dataset:load_dataset_pickle:337 - Датасет загружен из /home/developer/workspace/projects/3d_recognition_analisis/data/interim/dataset_metadata.pkl
Пример записи: DataModel(model_id='44. Extractor Pin-06', model_path='/home/developer/workspace/projects/3d_recognition_analisis/data/raw/3D/44. Extractor Pin/44. Extractor Pin-06.prt.stp', detail_type='44. Extractor Pin', image_paths=[ImageData(image_id='44. Extractor Pin-06_left', image_path='/home/developer/workspace/projects/3d_recognition_analisis/data/raw/2D/44. Extractor Pin/44. Extractor Pin-06/44. Extractor Pin-06_Left.jpg', model_type='left'), ImageData(image_id='44. Extractor Pin-06_right', image_path='/home/developer/workspace/projects/3d_recognition_analisis/data/raw/2D/44. Extractor Pin/44. Extractor P

## 2. Извлечение признаков для всех моделей

Для каждой модели вычислим вектор признаков с помощью существующих функций. Сохраним результаты в массив для дальнейшего анализа.

In [9]:
import numpy as np

from src.features.brep import BrepExtractor

brep = BrepExtractor()


def extract_features_for_shape(shape, k_spec=16, K_rdf=256):
    # Триангуляция
    V, F = brep._get_vertices_and_faces(shape)
    if V.shape[0] == 0 or F.shape[0] == 0:
        return None

    # Базовые признаки по триангуляции
    E, L = brep.mesh_edge_data(V, F)
    A = brep.mesh_face_areas(V, F)
    D = brep._mesh_dihedral_angles(V, F)
    deg = brep.mesh_vertex_degrees(F, V.shape[0])

    h_edge = brep.hist_norm(L, bins=64, log=True)
    h_area = brep.hist_norm(A, bins=64, log=True)
    h_dih  = brep.hist_norm(D, bins=64, rng=(0, np.pi))
    h_deg  = brep.hist_norm(deg, bins=32, rng=(0, deg.max() if deg.size else 1))

    # RDF
    rdf = brep._compute_rdf(V, F, K=K_rdf)

    # B-Rep признаки
    brep_vec = brep._brep_surface_type_hist(shape)

    # Спектральные признаки
    try:
        evals, _ = brep._compute_lbo_spectrum(V, F, k=k_spec, scale_invariant=True)
        padded_evals = np.zeros(k_spec, dtype=np.float32)
        num_evals = min(len(evals), k_spec)
        padded_evals[:num_evals] = evals[:num_evals]
    except Exception as e:
        padded_evals = np.zeros(k_spec, dtype=np.float32)

    # Объединяем все признаки в один вектор
    feature_vec = np.concatenate([
        rdf,            # 256
        h_edge,         # 64
        h_area,         # 64
        h_dih,          # 64
        h_deg,          # 32
        brep_vec,       # 10
        padded_evals    # 16
    ]).astype(np.float32)
    return feature_vec

# Извлечение признаков для всех моделей
features = []
failed_indices = []

for model in tqdm(dataset):
    shape = brep._load_step_shape(model.model_path)
    failed_indices.append(model.detail_type)
    feat = extract_features_for_shape(shape)
    failed_indices.append(model.detail_type)
    features.append(feat)

print(failed_indices)
# print("Размерность матрицы признаков:", features.shape)
# if failed_indices:
#     print(f"Не удалось обработать {len(failed_indices)} моделей: индексы {failed_indices}")

2025-08-12 14:25:39.539 | INFO     | src.features.brep:__init__:42 - Инициализация 'BRep' экстрактора. Размерность вектора: 506


100%|██████████| 129/129 [00:08<00:00, 15.20it/s]

['44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '44. Extractor Pin', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', '43. Extractor', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', 'Кожух', '

## 3. Анализ распределения каждого признака

Построим гистограммы и boxplot для каждого признака по всему датасету, чтобы выявить константные или аномальные признаки.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Для наглядности ограничим количество признаков для отображения
num_features_to_plot = min(20, features.shape[1])

plt.figure(figsize=(16, 8))
for i in range(num_features_to_plot):
    plt.subplot(4, 5, i+1)
    sns.histplot(features[:, i], bins=32, kde=False)
    plt.title(f'Feature {i}')
    plt.tight_layout()
plt.suptitle("Гистограммы первых 20 признаков", y=1.02)
plt.show()

# Boxplot для всех признаков (может быть полезно для поиска выбросов/констант)
plt.figure(figsize=(16, 4))
sns.boxplot(data=features[:, :num_features_to_plot])
plt.title("Boxplot первых 20 признаков")
plt.show()

# Проверим, есть ли константные признаки
const_mask = np.all(features == features[0, :], axis=0)
print(f"Константных признаков: {const_mask.sum()} из {features.shape[1]}")

## 4. Оценка корреляции между признаками

Вычислим корреляционную матрицу между признаками и визуализируем её с помощью heatmap, чтобы найти дублирующие признаки.

In [ ]:
# Корреляционная матрица (используем только не-константные признаки)
nonconst_features = features[:, ~const_mask]
corr = np.corrcoef(nonconst_features, rowvar=False)

plt.figure(figsize=(12, 10))
sns.heatmap(corr, cmap='coolwarm', center=0, square=True, cbar_kws={'shrink': 0.5})
plt.title("Корреляционная матрица между признаками (без константных)")
plt.show()

# Найдём пары признаков с высокой корреляцией
high_corr = np.where((np.abs(corr) > 0.95) & (np.abs(corr) < 1.0))
high_corr_pairs = list(zip(high_corr[0], high_corr[1]))
print(f"Пары признаков с корреляцией > 0.95: {high_corr_pairs[:10]}")

## 5. Проверка информативности признаков с помощью простых моделей

Обучим простую модель (например, классификатор или регрессор) на признаках, оценим важность признаков (feature importance) и качество модели.

**Примечание:** Для примера будем использовать случайную целевую переменную (или, если в датасете есть метки — используйте их).

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score

# Пример: если у вас есть метки классов, используйте их вместо random_targets
# random_targets = np.random.randint(0, 2, size=features.shape[0])
# y = [item.label for item in dataset if ...]  # если есть метки

# Для примера создадим случайную задачу классификации
random_targets = np.random.randint(0, 2, size=features.shape[0])

X_train, X_test, y_train, y_test = train_test_split(features, random_targets, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy (random targets): {acc:.3f}")

# Важность признаков
importances = clf.feature_importances_

## 6. Визуализация важности признаков

Построим график важности признаков, чтобы наглядно увидеть вклад каждого признака.

In [ ]:
# Визуализация важности признаков
sorted_idx = np.argsort(importances)[::-1]
top_n = 20

plt.figure(figsize=(12, 6))
plt.bar(range(top_n), importances[sorted_idx[:top_n]])
plt.xticks(range(top_n), [f'Feature {i}' for i in sorted_idx[:top_n]], rotation=45)
plt.title("Топ-20 важных признаков (Random Forest)")
plt.ylabel("Feature Importance")
plt.tight_layout()
plt.show()

---

**Выводы:**  
В этом ноутбуке мы провели базовый анализ информативности признаков 3D моделей: изучили их распределение, корреляцию, а также оценили их значимость с помощью простой модели. Такой анализ позволяет выявить неинформативные, дублирующие или аномальные признаки и улучшить качество последующих моделей.